<a href="https://colab.research.google.com/github/VidalsHugo/RecSys/blob/main/Recsys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [3]:
#UTILIZANDO DATASET MOVIELENS100K

# Carrega o arquivo CSV para um DataFrame do Pandas
df = pd.read_csv('u.data', sep='\t', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'])
# Descarta a coluna de timestamp
df.drop('timestamp', axis=1, inplace=True)
# Pivotagem dos dados para obter uma matriz de usuário-item
matriz_user_item = df.pivot(index='user_id', columns='movie_id', values='rating')
# Preenche os valores NaN com 0, indicando que o usuário não avaliou o item
matriz_user_item.fillna(0, inplace=True)

print(matriz_user_item.iloc[:, 70:80])

movie_id   71   72   73   74   75   76   77   78   79   80
user_id                                                   
1         3.0  4.0  3.0  1.0  4.0  4.0  4.0  1.0  4.0  4.0
2         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
3         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
4         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
5         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  3.0  2.0
...       ...  ...  ...  ...  ...  ...  ...  ...  ...  ...
939       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
940       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
941       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
942       5.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  5.0  0.0
943       0.0  2.0  3.0  0.0  0.0  4.0  0.0  0.0  5.0  2.0

[943 rows x 10 columns]


In [5]:
def k_vizinhos_mais_proximos(matriz_user_item, user, rating_usuario, k):
    vizinhos_mais_proximos = []
    for vizinho in matriz_user_item.index:
        if vizinho != user:
            rating_vizinho = matriz_user_item.loc[vizinho].values

            # Calculo da similaridade
            produto_escalar = rating_usuario @ rating_vizinho
            norma_usuario = np.linalg.norm(rating_usuario)
            norma_vizinho = np.linalg.norm(rating_vizinho)
            produto_normas = norma_usuario * norma_vizinho
            similaridade = round(produto_escalar / produto_normas, 3) if produto_normas != 0 else 0
            vizinhos_mais_proximos.append((vizinho, similaridade))

    vizinhos_mais_proximos.sort(key=lambda x: x[1], reverse=True)
    k_vizinhos_mais_proximos = vizinhos_mais_proximos[:k]
    return k_vizinhos_mais_proximos

def similaridade_cosseno(matriz_user_item, user, k, peso_similaridade):
    rating_usuario = matriz_user_item.loc[user].values
    vizinhos_mais_proximos = []
    indices_itens_nao_avaliados = matriz_user_item.columns[rating_usuario == 0.0]

    #print(f"\nIndices_itens_nao_avaliados: {indices_itens_nao_avaliados}")

    nota_prevista = []

    k_vizinhos_proximos = k_vizinhos_mais_proximos(matriz_user_item, user, rating_usuario, k)

    for i in indices_itens_nao_avaliados:
        somatorio_nota_sing = 0
        somatorio_sing = 0
        for vizinho, similaridade in k_vizinhos_proximos:
            nota_vizinho = matriz_user_item.loc[vizinho, i]
            somatorio_nota_sing += nota_vizinho * similaridade * peso_similaridade
            somatorio_sing += abs(similaridade * peso_similaridade)

        nota_prevista.append(((somatorio_nota_sing / somatorio_sing) if somatorio_sing != 0 else 0, i))

    nota_prevista.sort(reverse=True, key=lambda x: x[0])
    #print(f"\nNota_prevista: {nota_prevista}")
    print(f"\nVizinhos_mais_proximos: {k_vizinhos_proximos}")
    nota_prevista.sort(reverse=True)
    print(f"\nFilmes recomendados: ")
    for i, filme in enumerate(nota_prevista[:10]):
        print(f"{i+1}# \t{filme[1]}")

# Exemplo de chamada com o novo hiperparâmetro
similaridade_cosseno(matriz_user_item, 1, 10, peso_similaridade=1.5)



Vizinhos_mais_proximos: [(916, 0.569), (864, 0.548), (268, 0.542), (92, 0.541), (435, 0.539), (457, 0.538), (738, 0.527), (303, 0.526), (429, 0.526), (276, 0.525)]

Filmes recomendados: 
1# 	318
2# 	474
3# 	655
4# 	423
5# 	403
6# 	357
7# 	433
8# 	385
9# 	568
10# 	470
